# Tutorial 18: Agent Swarm/Network

In this tutorial, you'll build a decentralized agent swarm where agents communicate peer-to-peer without a central supervisor.

**What you'll learn:**
- How to create a swarm of agents with network topology
- Implementing peer-to-peer agent communication
- Dynamic routing where agents decide their successors
- Shared context management across the swarm
- Difference between swarm and supervisor patterns

By the end, you'll have a working agent swarm where researchers, analysts, and writers collaborate dynamically without central coordination.

## Prerequisites

- Completed Tutorials 14-16 (Multi-Agent Patterns)
- Understanding of multi-agent collaboration and routing
- Ollama running with a capable model (llama3.1:8b or larger recommended)

## Why Agent Swarms?

While supervisor patterns work well for structured tasks, swarms excel at:

1. **Decentralized Decision-Making**: No bottleneck through a central supervisor
2. **Flexible Collaboration**: Agents adapt based on peer outputs
3. **Scalability**: Add agents without modifying supervisor logic
4. **Emergent Behavior**: Complex patterns emerge from simple peer interactions

**Swarm vs Supervisor:**

```
Supervisor Pattern:                  Swarm Pattern:
                                    
    ┌─────────────┐                      ┌─────────┐
    │ Supervisor  │                      │ Agent A │
    └──────┬──────┘                      └────┬────┘
           │                                  │
    ┌──────┼──────┐                    ┌──────┼──────┐
    │      │      │                    │      │      │
    ▼      ▼      ▼                    ▼      ▼      ▼
  Agent  Agent  Agent                Agent  Agent  Agent
    A      B      C                    B      C      D
                                             │      │
  Central control                            └──────┘
  All routes through supervisor          Peer-to-peer
```

## Step 1: Setup and Verify Environment

In [ ]:
# Verify our setup
from langgraph_ollama_local import LocalAgentConfig

config = LocalAgentConfig()
print(f"Ollama URL: {config.ollama.base_url}")
print(f"Model: {config.ollama.model}")
print("Setup verified!")

## Step 2: Define the Swarm State

Swarm state differs from supervisor state:
- No `next_agent` set by supervisor
- `agents_state` tracks per-agent information
- `shared_context` enables all agents to see each other's work
- `current_agent` is set by the previous agent (peer routing)

In [ ]:
from typing import Annotated, Any
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages
import operator


class SwarmState(TypedDict):
    """State schema for agent swarm."""
    
    # Conversation history
    messages: Annotated[list, add_messages]
    
    # The task for the swarm
    task: str
    
    # Per-agent state dictionary
    agents_state: dict[str, dict[str, Any]]
    
    # Shared context from all agents - uses operator.add to accumulate
    shared_context: Annotated[list[dict], operator.add]
    
    # Current agent (set by previous agent, not supervisor)
    current_agent: str
    
    # Iteration tracking
    iteration: int
    max_iterations: int
    
    # Final result
    final_result: str


print("Swarm state defined!")
print("\nKey differences from supervisor pattern:")
print("- agents_state: Per-agent data (not centralized)")
print("- shared_context: All agents see all outputs")
print("- current_agent: Set by peers, not supervisor")

## Step 3: Define Swarm Agents with Connections

Each agent has:
- A role (system prompt)
- **Connections**: List of agents it can hand off to
- Optional tools

This defines the network topology!

In [ ]:
from pydantic import BaseModel, Field


class SwarmAgent(BaseModel):
    """Configuration for a swarm agent."""
    
    name: str = Field(description="Unique name for the agent")
    system_prompt: str = Field(description="System prompt defining role")
    connections: list[str] = Field(
        default_factory=list,
        description="List of agent names this agent can hand off to"
    )
    tools: list | None = Field(default=None, description="Optional tools")


# Define our research swarm
swarm_agents = [
    SwarmAgent(
        name="researcher",
        system_prompt="""You are a research specialist in a collaborative swarm.
Your role: Gather information, identify key topics, and provide initial findings.
Pass work to analysts or fact-checkers when you have initial findings.""",
        connections=["analyst", "fact_checker"],  # Can route to either
    ),
    SwarmAgent(
        name="analyst",
        system_prompt="""You are an analyst in a collaborative swarm.
Your role: Analyze research findings, identify patterns, draw insights.
Pass work to writers when analysis is complete.""",
        connections=["writer"],  # Routes to writer
    ),
    SwarmAgent(
        name="fact_checker",
        system_prompt="""You are a fact-checker in a collaborative swarm.
Your role: Verify claims, check for consistency, flag uncertainties.
Pass work to analysts or writers after verification.""",
        connections=["analyst", "writer"],  # Can route to either
    ),
    SwarmAgent(
        name="writer",
        system_prompt="""You are a writer in a collaborative swarm.
Your role: Synthesize all findings into a clear, comprehensive report.
You are typically the final agent - respond DONE when report is complete.""",
        connections=[],  # Terminal node (no outgoing connections)
    ),
]

print("Swarm agents defined!")
print("\nNetwork topology:")
for agent in swarm_agents:
    conns = ", ".join(agent.connections) if agent.connections else "[terminal]"
    print(f"  {agent.name} -> {conns}")

## Step 4: Create Swarm Agent Nodes

Each swarm agent:
1. Sees shared context from all previous agents
2. Performs its work
3. Decides which connected agent should go next
4. Shares its output with the swarm

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage


class SwarmRouting(BaseModel):
    """Routing decision from a swarm agent."""
    
    next_agent: str = Field(
        description="Name of next agent to work, or 'DONE' if complete"
    )
    reasoning: str = Field(
        description="Brief explanation for this routing decision"
    )
    share_context: bool = Field(
        default=True,
        description="Whether to share this output with the swarm"
    )


def create_swarm_node(llm, agent_config: SwarmAgent):
    """Create a node for a swarm agent."""
    
    # Use structured output for routing
    structured_llm = llm.with_structured_output(SwarmRouting)
    agent_llm = llm.bind_tools(agent_config.tools) if agent_config.tools else llm
    
    def swarm_agent_node(state: SwarmState) -> dict:
        """Execute the swarm agent's work and routing."""
        
        # Build context from shared findings
        shared_context = state.get("shared_context", [])
        if shared_context:
            context_parts = [
                f"**{ctx['agent']}**: {ctx['content']}"
                for ctx in shared_context
            ]
            context_str = "\n\n".join(context_parts)
        else:
            context_str = "No shared context yet. You are the first agent."
        
        # Get agent's work
        work_messages = [
            SystemMessage(content=agent_config.system_prompt),
            HumanMessage(content=f"""Task: {state['task']}

Shared Context from Swarm:
{context_str}

Provide your contribution to the task.""")
        ]
        
        work_response = agent_llm.invoke(work_messages)
        agent_output = work_response.content
        
        # Get routing decision
        connections_str = ", ".join(agent_config.connections) if agent_config.connections else "none"
        routing_messages = [
            SystemMessage(content=f"""You are {agent_config.name} in a swarm network.

Your connections: {connections_str}

Decide which connected agent should work next (or 'DONE' if task is complete)."""),
            HumanMessage(content=f"""Your work:
{agent_output}

Task: {state['task']}
Iteration: {state['iteration'] + 1}/{state['max_iterations']}

Which agent should work next?""")
        ]
        
        routing_decision = structured_llm.invoke(routing_messages)
        
        # Update state
        agents_state = state.get("agents_state", {}).copy()
        agents_state[agent_config.name] = {
            "last_output": agent_output,
            "routing_decision": routing_decision.next_agent,
            "work_count": agents_state.get(agent_config.name, {}).get("work_count", 0) + 1,
        }
        
        updates = {
            "agents_state": agents_state,
            "current_agent": routing_decision.next_agent,
            "iteration": state["iteration"] + 1,
            "messages": [AIMessage(
                content=f"[{agent_config.name}] Next: {routing_decision.next_agent}. {routing_decision.reasoning}"
            )],
        }
        
        # Share context if requested
        if routing_decision.share_context:
            updates["shared_context"] = [{
                "agent": agent_config.name,
                "content": agent_output,
                "iteration": state["iteration"] + 1,
            }]
        
        return updates
    
    return swarm_agent_node


print("Swarm node creator defined!")
print("\nKey feature: Each agent decides its own successor!")

## Step 5: Create the Aggregate Node

This node combines all swarm outputs into a final result.

In [ ]:
def aggregate_node(state: SwarmState) -> dict:
    """Aggregate all swarm agent outputs."""
    
    shared_context = state.get("shared_context", [])
    
    if not shared_context:
        return {"final_result": "No work was completed by the swarm."}
    
    # Group by agent
    by_agent = {}
    for ctx in shared_context:
        agent = ctx["agent"]
        if agent not in by_agent:
            by_agent[agent] = []
        by_agent[agent].append(ctx)
    
    # Build result
    result_parts = ["# Agent Swarm Results\n"]
    
    for agent_name, contexts in by_agent.items():
        result_parts.append(f"## {agent_name.title()}")
        for i, ctx in enumerate(contexts, 1):
            if len(contexts) > 1:
                result_parts.append(f"### Iteration {ctx['iteration']}")
            result_parts.append(ctx["content"])
            result_parts.append("")  # Empty line
    
    return {
        "final_result": "\n\n".join(result_parts).strip(),
        "messages": [AIMessage(content="[System] Swarm work aggregated.")]
    }


print("Aggregate node defined!")

## Step 6: Define Swarm Routing

Unlike supervisor pattern with a central router, swarm routing simply reads the `current_agent` field set by the previous agent.

In [ ]:
def route_swarm(state: SwarmState) -> str:
    """Route to the next agent based on peer decision."""
    
    current_agent = state.get("current_agent", "")
    iteration = state.get("iteration", 0)
    max_iterations = state.get("max_iterations", 10)
    
    # Force completion at max iterations
    if iteration >= max_iterations:
        print(f"  [Swarm Router] Max iterations reached, aggregating")
        return "aggregate"
    
    # Route based on agent's decision
    if current_agent == "DONE" or not current_agent:
        print(f"  [Swarm Router] Task complete, aggregating")
        return "aggregate"
    
    print(f"  [Swarm Router] Routing to {current_agent}")
    return current_agent


print("Swarm routing defined!")
print("\nKey difference: No central decision-maker!")
print("Each agent sets current_agent for the next peer.")

## Step 7: Build the Swarm Graph

Now we assemble the swarm network!

In [ ]:
from langgraph.graph import StateGraph, START, END
from langchain_ollama import ChatOllama

# Create LLM
llm = ChatOllama(
    model=config.ollama.model,
    base_url=config.ollama.base_url,
    temperature=0,
)

# Build the graph
workflow = StateGraph(SwarmState)

# Add agent nodes
for agent in swarm_agents:
    node = create_swarm_node(llm, agent)
    workflow.add_node(agent.name, node)

# Add aggregate node
workflow.add_node("aggregate", aggregate_node)

# Entry point: start at first agent (researcher)
workflow.add_edge(START, "researcher")

# Build routing map (all agents + aggregate)
routing_map = {agent.name: agent.name for agent in swarm_agents}
routing_map["aggregate"] = "aggregate"

# All agents route dynamically
for agent in swarm_agents:
    workflow.add_conditional_edges(
        agent.name,
        route_swarm,
        routing_map,
    )

# Aggregate leads to end
workflow.add_edge("aggregate", END)

# Compile
graph = workflow.compile()

print("Swarm graph compiled!")
print("\nGraph structure:")
print("  START -> researcher")
print("  researcher -> [analyst | fact_checker | aggregate]")
print("  analyst -> [writer | aggregate]")
print("  fact_checker -> [analyst | writer | aggregate]")
print("  writer -> aggregate")
print("  aggregate -> END")
print("\nNote: Routing is DYNAMIC - decided by each agent!")

## Step 8: Run the Swarm

Let's test our swarm with a research task!

In [ ]:
def run_swarm_task(task: str, max_iterations: int = 8):
    """Run a task through the agent swarm."""
    
    print("="*60)
    print(f"Task: {task}")
    print("="*60)
    
    initial_state = {
        "messages": [HumanMessage(content=f"Task: {task}")],
        "task": task,
        "agents_state": {},
        "shared_context": [],
        "current_agent": "",
        "iteration": 0,
        "max_iterations": max_iterations,
        "final_result": "",
    }
    
    result = graph.invoke(initial_state)
    
    print("\n" + "="*60)
    print("FINAL RESULT")
    print("="*60)
    print(result["final_result"])
    print("\n" + "="*60)
    print(f"Completed in {result['iteration']} iterations")
    print(f"Agents that contributed: {len(result['shared_context'])}")
    
    # Show agent work counts
    print("\nAgent work summary:")
    for agent_name, agent_data in result["agents_state"].items():
        work_count = agent_data.get("work_count", 0)
        print(f"  {agent_name}: {work_count} contribution(s)")
    
    return result


# Test with a research task
result = run_swarm_task(
    "Research the key benefits and challenges of using LangGraph for multi-agent systems"
)

## Step 9: Observe Swarm Dynamics

Let's visualize how the swarm collaborated:

In [ ]:
def visualize_swarm_flow(result):
    """Visualize the agent collaboration flow."""
    
    print("\nSwarm Collaboration Flow:")
    print("="*60)
    
    shared_context = result.get("shared_context", [])
    
    for i, ctx in enumerate(shared_context, 1):
        agent = ctx["agent"]
        iteration = ctx.get("iteration", 0)
        content_preview = ctx["content"][:100] + "..."
        
        print(f"\n{i}. [{iteration}] {agent.upper()}")
        print(f"   {content_preview}")
        
        # Show routing decision
        agent_state = result["agents_state"].get(agent, {})
        next_agent = agent_state.get("routing_decision", "unknown")
        print(f"   -> Routed to: {next_agent}")


visualize_swarm_flow(result)

## Step 10: Try Different Tasks

The swarm adapts to different task types!

In [ ]:
# Technical analysis task
result2 = run_swarm_task(
    "Analyze the tradeoffs between microservices and monolithic architectures",
    max_iterations=6
)

visualize_swarm_flow(result2)

In [ ]:
# Fact-checking focused task
result3 = run_swarm_task(
    "Research claims about quantum computing breaking current encryption",
    max_iterations=7
)

visualize_swarm_flow(result3)

## Step 11: Using the Built-in Module

The `langgraph_ollama_local.patterns` module provides a ready-to-use implementation:

In [ ]:
from langgraph_ollama_local.patterns import (
    SwarmAgent,
    create_swarm_graph,
    run_swarm_task,
)

# Define agents using the module
module_agents = [
    SwarmAgent(
        name="researcher",
        system_prompt="Research and gather information",
        connections=["analyst"],
    ),
    SwarmAgent(
        name="analyst",
        system_prompt="Analyze and draw insights",
        connections=["writer"],
    ),
    SwarmAgent(
        name="writer",
        system_prompt="Write final report",
        connections=[],
    ),
]

# Create and run swarm
module_graph = create_swarm_graph(llm, module_agents)

result = run_swarm_task(
    module_graph,
    "Analyze the impact of AI on software development",
    max_iterations=6
)

print("Final Result:")
print(result["final_result"])

## Complete Code

Here's the complete implementation in one cell for reference:

In [ ]:
# === Complete Agent Swarm Implementation ===

from typing import Annotated, Any
from typing_extensions import TypedDict
import operator

from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_ollama import ChatOllama
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from pydantic import BaseModel, Field

from langgraph_ollama_local import LocalAgentConfig


# === State ===
class SwarmState(TypedDict):
    messages: Annotated[list, add_messages]
    task: str
    agents_state: dict[str, dict[str, Any]]
    shared_context: Annotated[list[dict], operator.add]
    current_agent: str
    iteration: int
    max_iterations: int
    final_result: str


# === Agent Config ===
class SwarmAgent(BaseModel):
    name: str
    system_prompt: str
    connections: list[str] = Field(default_factory=list)


class SwarmRouting(BaseModel):
    next_agent: str
    reasoning: str
    share_context: bool = True


# === Node Creator ===
def create_swarm_node(llm, agent_config: SwarmAgent):
    structured_llm = llm.with_structured_output(SwarmRouting)
    
    def node(state):
        context = "\n".join([f"{c['agent']}: {c['content'][:200]}" for c in state.get("shared_context", [])])
        work = llm.invoke([SystemMessage(content=agent_config.system_prompt),
                          HumanMessage(content=f"Task: {state['task']}\nContext: {context}")])
        
        routing = structured_llm.invoke([SystemMessage(content=f"Connections: {', '.join(agent_config.connections)}"),
                                        HumanMessage(content=f"Your work: {work.content}\nWhich agent next?")])
        
        agents_state = state.get("agents_state", {}).copy()
        agents_state[agent_config.name] = {"last_output": work.content, "routing_decision": routing.next_agent}
        
        return {
            "agents_state": agents_state,
            "current_agent": routing.next_agent,
            "iteration": state["iteration"] + 1,
            "shared_context": [{"agent": agent_config.name, "content": work.content}] if routing.share_context else [],
        }
    return node


def aggregate(state):
    contexts = state.get("shared_context", [])
    result = "\n\n".join([f"## {c['agent'].title()}\n{c['content']}" for c in contexts])
    return {"final_result": result}


def route_swarm(state):
    if state["iteration"] >= state["max_iterations"] or state["current_agent"] == "DONE":
        return "aggregate"
    return state["current_agent"]


# === Build Graph ===
def build_swarm_graph(agents: list[SwarmAgent]):
    config = LocalAgentConfig()
    llm = ChatOllama(model=config.ollama.model, base_url=config.ollama.base_url, temperature=0)
    
    g = StateGraph(SwarmState)
    for agent in agents:
        g.add_node(agent.name, create_swarm_node(llm, agent))
    g.add_node("aggregate", aggregate)
    
    g.add_edge(START, agents[0].name)
    routing_map = {a.name: a.name for a in agents}
    routing_map["aggregate"] = "aggregate"
    for agent in agents:
        g.add_conditional_edges(agent.name, route_swarm, routing_map)
    g.add_edge("aggregate", END)
    
    return g.compile()


# === Use ===
if __name__ == "__main__":
    agents = [
        SwarmAgent(name="researcher", system_prompt="Research", connections=["writer"]),
        SwarmAgent(name="writer", system_prompt="Write", connections=[]),
    ]
    graph = build_swarm_graph(agents)
    result = graph.invoke({
        "messages": [], "task": "Analyze AI trends",
        "agents_state": {}, "shared_context": [],
        "current_agent": "", "iteration": 0,
        "max_iterations": 5, "final_result": ""
    })
    print(result["final_result"])

## Key Concepts Recap

| Concept | Description |
|---------|-------------|
| **Decentralized Routing** | Each agent decides its successor, no central supervisor |
| **Network Topology** | Connections define which agents can communicate |
| **Shared Context** | All agents see accumulated work from the swarm |
| **Peer-to-Peer** | Agents hand off directly to connected peers |
| **Dynamic Adaptation** | Routing decisions based on current state and outputs |
| **Emergent Behavior** | Complex collaboration emerges from simple rules |
| **Per-Agent State** | Each agent maintains its own state history |

## Swarm vs Supervisor: When to Use Each

**Use Swarm Pattern when:**
- Tasks benefit from flexible, adaptive routing
- No clear hierarchical structure exists
- Agents need rich context from all peers
- You want emergent collaboration patterns
- Decentralized decision-making is preferred

**Use Supervisor Pattern when:**
- Clear task structure and agent roles
- Centralized control and monitoring needed
- Predictable routing is important
- Quality control via central oversight
- Simpler to reason about and debug

**Hybrid Approach:**
- Use hierarchical teams where each team is a swarm!
- Combine supervisor coordination with swarm sub-teams

## What's Next

You've completed the Multi-Agent Patterns phase! Key patterns learned:

- **Tutorial 14**: Supervisor pattern for centralized coordination
- **Tutorial 15**: Hierarchical teams with nested supervisors
- **Tutorial 16**: Subgraph patterns for modular composition
- **Tutorial 18**: Agent swarms for decentralized collaboration

These patterns can be combined to build sophisticated multi-agent systems!